In [4]:
using Polynomials
using Plots
using Jacobi
using LinearAlgebra
using SparseArrays

In [22]:
function basis_function(N,d,x,k,rc)
    basis_mat = zeros(N,N)*1.0
        for i=1:N
            for j=1:N
                Prod_temp=1
                for z=1:d
                    Prod_temp*=chebyshev(x[i,z],k[j,z])
                end
                basis_mat[i,j]=Prod_temp
            end
        end
        det_mat=det(basis_mat)
        boundary_func=vcat(map(x->rc^2-x^2, x))
        func_value=det_mat*prod(boundary_func)
    return basis_mat,func_value,det_mat,prod(boundary_func)
end

basis_function (generic function with 1 method)

In [23]:
function d2chebyshev(d2chebyshev_x,d2chebyshev_k)
    chebyshev_Tk=chebyshev(d2chebyshev_x, d2chebyshev_k)
    chebyshev_Uk=chebyshev2(d2chebyshev_x, d2chebyshev_k)
    if d2chebyshev_x!=1&-1
        d2_Tk=d2chebyshev_k*((d2chebyshev_k+1)*chebyshev_Tk-chebyshev_Uk)/(d2chebyshev_x^2-1)
    end
    if d2chebyshev_x==1
        d2_Tk=(d2chebyshev_k^4-d2chebyshev_k^2)/3
    end
    if d2chebyshev_x==-1
        d2_Tk=(-1)^k*(d2chebyshev_k^4-d2chebyshev_k^2)/3
    end
    return d2_Tk
end

d2chebyshev (generic function with 1 method)

In [30]:
function laplace_psi(N,d,x,k,rc)
    Sum_laplace=0
    Prod_laplace=0
    Sum_laplace=0
    Prod_laplace=0
    basis_func=basis_function(N,d,x,k,rc)
    basis_mat=basis_func[1]
    basisfunc=basis_func[2]
    det_mat=basis_func[3]
    prod_boundary_func=basis_func[4]
        for dimension=1:d
            for N_x=1:N
                    mat1=basis_func[1]
                    mat2=basis_func[1]
                    for i=1:N
                        #################d2T####################   
                        d2T_k=d2chebyshev(x[N_x,dimension], k[i,dimension])
                        T_k=Jacobi.chebyshev(x[N_x,dimension], k[i,dimension])
                        mat1[N_x,i]=basis_function(N,d,x,k,rc)[1][N_x,i]*d2T_k/T_k
                        dT_ki=Jacobi.dchebyshev(x[N_x,dimension], k[i,dimension])
                        T_k_i=Jacobi.chebyshev(x[N_x,dimension], k[i,dimension])
                        mat2[N_x,i]=basis_function(N,d,x,k,rc)[1][N_x,i]*dT_ki/T_k_i
                    end
                    Sum_laplace+=det(mat1)*prod_boundary_func
                    Sum_laplace+=2*det(mat2)*prod_boundary_func*(-2*x[N_x,dimension])/(rc^2-x[N_x,dimension]^2)
                #################d2x####################
                Sum_laplace+=-2*basisfunc/(rc^2-x[N_x,dimension]^2)
            end
        end
    return Sum_laplace
end

laplace_psi (generic function with 1 method)

In [8]:
function V_psi(N,d,x,k,rc)
    Sum_V=0
    Sum_r=0
    for i=1:N*d
        Sum_r+=x[i]^2
    end
        Sum_V=basis_function(N,d,x,k,rc)[2]*Sum_r
    return Sum_V
end

V_psi (generic function with 1 method)

In [9]:
function H_psi(N,d,x,k,rc)
       Sum_H_psi=-1/2*laplace_psi(N,d,x,k,rc)+1/2*V_psi(N,d,x,k,rc)
    return Sum_H_psi
end

H_psi (generic function with 1 method)

In [10]:
using Combinatorics
using BenchmarkTools
function nsumk(N,M)
    k=zeros(Int64,1,N)
    k_temp=collect(partitions(M+N,N))
    k_temp=vcat(map(e->collect(e)', k_temp)...)
    k_temp=k_temp-ones(Int64,size(k_temp))
    k=vcat(k,k_temp)
    return k
end

nsumk (generic function with 1 method)

In [12]:
function find_k(d,N,M)
k_all=zeros(Int64,1,N*d)
    for i=1:M
        dividers=nsumk(N*d,i)
        k_all=vcat(k_all, dividers)
    end
k_all=sort!(k_all,dims =2)
    k_all=unique(k_all,dims=1)
    k_all=k_all'
k_all=reshape(k_all,N,d,size(k_all)[2])
k_temp=zeros(Int64,N,d,size(k_all)[3])
for i=1:size(k_all)[3]
    if size(unique(k_all[:,:,i],dims=1))[1]==N
        k_temp[:,:,i]=k_all[:,:,i]
    end
end
k_temp=unique(k_temp,dims=3)  
k_all=reshape(k_temp,N*d,size(k_temp)[3])
number_of_k=size(k_all)[2]
    k_temp=zeros(Int64,1,N*d)
    i=0
    k_temp=collect(permutations(k_all[1+i*N*d:N*d+i*N*d]))
    k_temp=vcat(map(e->collect(e)', k_temp)...)
    k=k_temp
    for i=1:number_of_k-1
        k_temp=collect(permutations(k_all[1+i*N*d:N*d+i*N*d]))
        k_temp=vcat(map(e->collect(e)', k_temp)...)
        k=vcat(k, k_temp)
        k=unique(k,dims=1)
    end
    K=k'
    number_of_k=size(k)[1]
    K=reshape(K,number_of_k*N*d)
    K=reshape(K,N,d,number_of_k)
    K_unique=zeros(Int64,size(K))
    for i=1:number_of_k
        K[:,:,i]=sortslices(K[:,:,i],dims=1)
        if size(unique(K[:,:,i],dims=1))[1]>1
            K_unique[:,:,i]=K[:,:,i]
        end
    end
        K=unique(K_unique,dims=3)  
        Number_k=size(K)[3]
        K=K[:,:,2:Number_k]

   return K
end

find_k (generic function with 1 method)

In [13]:
function random_choose(Number_of_x,d,N,k,rc)
    Number_of_k=size(k)[3]
    random_choose_X=rand(N,d,Number_of_x).*rand([-rc,rc],N,d,Number_of_x)
    random_choose_psi_x=zeros(Number_of_k,Number_of_x)
    random_choose_Hpsi_x=zeros(Number_of_k,Number_of_x)
    for i=1:Number_of_k
        for j=1:Number_of_x
            random_choose_psi_x[i,j]=basis_function(N,d,random_choose_X[:,:,j],k[:,:,i],range)[1]
            random_choose_Hpsi_x[i,j]=H_psi(N,d,random_choose_X[:,:,j],k[:,:,i],range)
        end
    end
    return random_choose_psi_x,random_choose_Hpsi_x,random_choose_X
end

random_choose (generic function with 1 method)

In [14]:
function least_squares(least_squares_x,least_squares_y)
    least_squares_C_k=inv(least_squares_x'*least_squares_x)*least_squares_x'*least_squares_y
    return least_squares_C_k
end

least_squares (generic function with 1 method)

In [38]:
function Eigenvalue(random_x,coef,alpha,d,N,k,rc,Number_of_x;eps_x = 10^(-20),eps_d=0.001,maxIterations = 100)
    record=spzeros(maxIterations)
    basisfunction=random_x[1]
    H_basisfunction=random_x[2]
    for t in 1:maxIterations
            reilaygh_quotientalpha=reilaygh_quotient_alpha(coef,basisfunction,H_basisfunction,N,d,rc,Number_of_x)
            reilaygh_quotient=reilaygh_quotientalpha[1]
            H_wave=reilaygh_quotientalpha[2]
            coef=reilaygh_quotientalpha[3]
            p=reilaygh_quotientalpha[4]
            coef_H_wave=reilaygh_quotientalpha[5]
            record[t]=reilaygh_quotient[1]
        if t>=3
            if abs(record[t]-record[t-1])+abs(record[t-1]-record[t-2])<=2*eps_d
                println("Convergence is reached after  ",t,"  iterations.")
                return coef,reilaygh_quotient,record
            end
        end
        alpha=0.001
        #steep(p,coef,reilaygh_quotient,coef_H_wave,basisfunction,N,d,range,H_basisfunction,Number_of_x,0,0.1,0.01,100)[1]
        coef = coef-alpha*p
        println("At step ", t, " and reilaygh quotient = ", reilaygh_quotient)
    end
    println("Warning:",maxIterations,"  iterations have been exceeded")
    return coef,reilaygh_quotient,record
end

Eigenvalue (generic function with 1 method)

In [39]:
function reilaygh_quotient_alpha(coef_temp,basisfunction,H_basisfunction,N,d,rc,Number_of_x)
        wavefunc=coef_temp*basisfunction
        sum_2_wavefunc=wavefunc*wavefunc'
        integal_wavefunc=(2*rc)^(N*d)*sum_2_wavefunc/Number_of_x
        coef_temp=coef_temp/sqrt(integal_wavefunc[1])
        wavefunc=wavefunc/sqrt(integal_wavefunc[1])
        sum_2_wavefunc=sum_2_wavefunc/integal_wavefunc[1]
    ########################################################
        H_wavefunc=coef_temp*H_basisfunction
        sum_wavefunc_Hwavefunc=wavefunc*H_wavefunc'
        reilaygh_quotient=sum_wavefunc_Hwavefunc/sum_2_wavefunc
        coef_H_wave=least_squares(basisfunction',H_wavefunc')
        coef_H_wave=coef_H_wave'
        p=coef_H_wave-reilaygh_quotient*coef_temp
    return reilaygh_quotient,H_wavefunc,coef_temp,p,coef_H_wave
end

reilaygh_quotient_alpha (generic function with 1 method)

In [40]:
d=1
N=3
M=10
@time k=find_k(d,N,M)
Number_of_x=40000
rc=5
@time random_x=random_choose(Number_of_x,d,N,k,rc)
Number_k=size(k)[3]
coef=ones(1,Number_k)
eps_x = 10^(-6)
eps_d=0.001
alpha=0.001
@time A=Eigenvalue(random_x,coef,alpha,d,N,k,rc,Number_of_x;eps_x= 10^(-30),eps_d=10^(-7),maxIterations = 100000)

  0.006892 seconds (13.33 k allocations: 1.322 MiB, 72.88% gc time)
 20.217265 seconds (231.88 M allocations: 23.044 GiB, 12.32% gc time)
At step 1 and reilaygh quotient = [16.764156182195528]
At step 2 and reilaygh quotient = [16.727175161692585]
At step 3 and reilaygh quotient = [16.690328076949374]
At step 4 and reilaygh quotient = [16.653616833994366]
At step 5 and reilaygh quotient = [16.617043278145935]
At step 6 and reilaygh quotient = [16.580609193909137]
At step 7 and reilaygh quotient = [16.544316304917384]
At step 8 and reilaygh quotient = [16.50816627391786]
At step 9 and reilaygh quotient = [16.47216070280002]
At step 10 and reilaygh quotient = [16.436301132666532]
At step 11 and reilaygh quotient = [16.400589043945732]
At step 12 and reilaygh quotient = [16.36502585654484]
At step 13 and reilaygh quotient = [16.329612930042938]
At step 14 and reilaygh quotient = [16.29435156392288]
At step 15 and reilaygh quotient = [16.25924299784121]
At step 16 and reilaygh quotient = [

([1.1885387959726657e-6 3.0813179713324065e-8 … 5.323881040776154e-14 -7.273191089990582e-14], [4.491466991840861],   [1     ]  =  16.7642
  [2     ]  =  16.7272
  [3     ]  =  16.6903
  [4     ]  =  16.6536
  [5     ]  =  16.617
  [6     ]  =  16.5806
  [7     ]  =  16.5443
  [8     ]  =  16.5082
  [9     ]  =  16.4722
  [10    ]  =  16.4363
            ⋮
  [2752  ]  =  4.49147
  [2753  ]  =  4.49147
  [2754  ]  =  4.49147
  [2755  ]  =  4.49147
  [2756  ]  =  4.49147
  [2757  ]  =  4.49147
  [2758  ]  =  4.49147
  [2759  ]  =  4.49147
  [2760  ]  =  4.49147
  [2761  ]  =  4.49147
  [2762  ]  =  4.49147)

In [42]:
##########################
#求 d=1 N=3 M=2:20 reilaygh quotient变化
#运行时间较长
##########################
d=1
N=3
M=10
k=find_k(d,N,M)
rc=5
Number_k=size(k)[3]
c_k=ones(1,Number_k)
eps_x = 10^(-6)
eps_d=0.001
alpha=0.001
Number_of_x=50000
record_time=zeros(1,30)
println("=================================M=",M,"==============================================")
#println("========================选点并计算基函数值时间如下","==============================================")
time = @elapsed random_x=random_choose(Number_of_x,d,N,k,rc)
record_time[1]=time
time = @elapsed dn=Eigenvalue(random_x,c_k,alpha,d,N,k,rc,Number_of_x;eps_x= 10^(-30),eps_d=10^(-7),maxIterations = 100000)
df1=dn[3]
record_time[2]=time
#println("========================梯度下降时间如上","==============================================")
t=1
err=1
df3=dn[2]
println("=================================df3==============================================")
print(df3)
while err>0.001
    M=M+2
    println("=================================M=",M,"==============================================")
    k=find_k(d,N,M)
    Number_k=size(k)[3]
    c_k=ones(1,Number_k)
    #println("========================选点并计算基函数值时间如下","==============================================")
    time = @elapsed random_x=random_choose(Number_of_x,d,N,k,rc)
    record_time[2*t+1]=time
    time = @elapsed dn=Eigenvalue(random_x,c_k,alpha,d,N,k,rc,Number_of_x;eps_x= 10^(-30),eps_d=10^(-7),maxIterations = 100000)
    df2=dn[3]  
    err=(df3-dn[2])[1]
    println("=================================err==============================================")
print(err)
    df3=dn[2]
    record_time[2*t+2]=time
    #println("========================梯度下降时间如上","==============================================")
    record=hcat(df1, df2)
    df1=record
    t=t+1
end

=================================M=10==============================================
At step 1 and reilaygh quotient = [16.74478503729003]
At step 2 and reilaygh quotient = [16.710293873599667]
At step 3 and reilaygh quotient = [16.675911165726326]
At step 4 and reilaygh quotient = [16.641638524320907]
At step 5 and reilaygh quotient = [16.60747751384905]
At step 6 and reilaygh quotient = [16.57342965245516]
At step 7 and reilaygh quotient = [16.539496411858096]
At step 8 and reilaygh quotient = [16.505679217277823]
At step 9 and reilaygh quotient = [16.471979447393164]
At step 10 and reilaygh quotient = [16.43839843432967]
At step 11 and reilaygh quotient = [16.404937463677506]
At step 12 and reilaygh quotient = [16.371597774538724]
At step 13 and reilaygh quotient = [16.338380559603255]
At step 14 and reilaygh quotient = [16.305286965253494]
At step 15 and reilaygh quotient = [16.272318091696306]
At step 16 and reilaygh quotient = [16.239474993122514]
At step 17 and reilaygh quotient 

LoadError: InterruptException: